In [ ]:
DESCRIPTION = """Using quantum espresso to compute the stresses associated with different strain states and fitting the elastic tensor."""

In [ ]:
# Import libraries
import numpy as np
from mp_api.client import MPRester
from pymatgen.io.pwscf import PWInput
from pymatgen.core import Structure
from pymatgen.analysis.elasticity import *
from pymatgen.io.vasp.inputs import *
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import os

from simtool import getValidatedInputs, DB

In [ ]:
%load_ext yamlmagic

In [ ]:
%%yaml INPUTS

mp_id:
    type: Text
    description: Materials Project ID for chosen structure
        
structure_dict:
    type: Dict
    description: Pymatgen structure object saved as dictionary

num_atoms:
    type: Integer
    description: Total number of atoms in the simulation cell
    min: 0
    max: 200
        
n_atom_types:
    type: Integer
    description: Number of elements in compostion. Examples Si - 1, TiN - 2, BaZrO3 - 3
        
space_group:
    type: Integer
    description: Space group number, i.e. 225
    value: 0
    min: 0
    max: 230
        
is_metal:
    type: Boolean
    description: Is the system considered assumed to be metallic or not. Used to include occupation information for the QE calculations.
    value: False
        
KE_cutoff:
    type: Number
    description: Kinetic energy cutoff (Ry)
    value: 50
    min: 10
    max: 300
    units: rydberg
        
k_x:
    type: Integer
    description: Number of k-points in x direction
    value: 1
    min: 1
    max: 30

k_y:
    type: Integer
    description: Number of k-points in y direction
    value: 1
    min: 1
    max: 30
    
k_z:
    type: Integer
    description: Number of k-points in z direction
    value: 1
    min: 1
    max: 30
        
i_steps:
    type: Integer
    description: Maximum number of ionic steps during relaxation
    value: 50
    
e_steps:
    type: Integer
    description: Maximum number of electronic steps during SCF calculation
    value: 100
    min: 100
    max: 1000
        
scf_conv:
    type: Number
    description: Convergence threshold for selfconsistency
    value: 1e-10
        
energy_conv:
    type: Number
    description: Convergence threshold on total energy for ionic minimization
    value: 1e-10
    min: 1e-15
    max: 1e-4
        
force_conv:
    type: Number
    description: Convergence threshold on forces for ionic minimization 
    value: 1e-5
    min: 1e-8
    max: 1e-3
        
vdw_corr:
    type: Choice
    description: Type of vdw correction. Options are 'grimme-d2', 'grimme-d3', or 'none'.
    options: ['grimme-d2','grimme-d3','none']
    value: 'none'
        
spin_polar:
    type: Boolean
    description: This parameter controls whether a spin-polarized calculation is run. False for non-polarized and True for spin-polarized, LSDA (magnetization along z axis)
    value: False
        
hubbard_U_on:
    type: Boolean
    description: This parameter controls whether a the Hubbard U correction is added to the calculation to account for localized d or f electrons that are not well described by standard DFT. False for no Hubbard U correction and True for adding the Hubbard U correction
    value: False
        
hubbard_projector:
    type: Choice
    description: This will determine the type of Hubbard projector used in the calculation. Options are 'atomic', 'ortho-atomic', 'norm-atomic', 'wf', or 'pseudo'. If the hubbard_U_on input is set to False this input is ignored when creating the QE input file. It is highly recommended to use ortho-atomic or atomic whenever possbile.
    options: ['atomic','ortho-atomic','norm-atomic','wf','pseudo']
    value: 'ortho-atomic'
        
hubbard_U_values:
    type: Dict
    description: This is a dictionary that will have the value for the U parameter in eV for each atomic species. 
    
hubbard_U_orbitals:
    type: Dict
    description: This is a dictionary that will have the orbitals that the U parameter is acting on for each atomic species.

pseudo:
    type: Choice
    description: Controls the type of pseudopotential used for the calculation
    options: ['PAW','USPP']
    value: 'PAW'
        
strain_matrices:
    type: List
    description: List containing the pymatgen Strain objects that will be used to generate the deformed structures and fit the elastic tensor

In [ ]:
defaultInputs = getValidatedInputs(INPUTS)
if defaultInputs:
    globals().update(defaultInputs)

In [ ]:
EXTRA_FILES = ['pseudo']

In [ ]:
%%yaml OUTPUTS

deformed_structures:
    type: List
    description: List of pymatgen Structure objects that were generated by the applying strains to the pristine structure
        
stress_tensors:
    type: List
    description: List of pymatgen Stress objects that were computed by quantum espresso for each deformed structure and used in the fit for the elastic tensor
    
elastic_tensor:
    type: Array
    description: Elastic tensor fit with pymatgen using the Moore-Penrose pseudo-inverse method

In [ ]:
textOutput = textInput

In [ ]:
db = DB(OUTPUTS)

In [ ]:
db.save('textOutput', textOutput)